In [1]:
import numpy as np

In [2]:
class CreateDataset:
    """ create the 2 XOR datasets, X & Y, for the DNN model """
    
    # create the entry dataset X
    def create_X(self, X_size):
        self.X = np.random.randint(2, size=(2, X_size))
        return self.X
    
    # create the label dataset Y
    def create_Y(self, X):
        self.Y = np.sum(X, axis=0)
        self.Y[self.Y != 1] = 0
        return self.Y

In [7]:
class Activation:
    """ the abstract class for all the activation funtion classes"""
    
    # the basic formula of the activation function for the forward pass
    def formula(self, Z):
        raise NotImplementdError
    
    # to calculate the derivative of the activation function for the backward pass
    def derivative(self, input):
        raise NotImplementdError
    
    # to be used to finetune the initialized weight according to the activation function set for the first layer
    def heuristic(self, layer_dims):
        raise NotImplementdError
    

class Sigmoid(Activation):
    """ all the functions related to the sigmoid activation function """
    
    # the basic formula of the sigmoid function for the forward pass
    def formula(self, Z):
        return 1 / (1 + np.exp(-Z))
    
    # to calculate the derivative of the sigmoid function for the backward pass
    def derivative(self, A):
        return A * (1 - A)
    
    # to be used to finetune the initialized weight if sigmoid function is set for the first layer
    def heuristic(self, layer_dims):
        return np.sqrt(1 / layer_dims)
    

class Tanh(Activation):
    """ all the functions related to the tanh activation function """
    
    # the basic formula of the tanh function for the forward pass
    def formula(self, Z):
        return (np.exp(Z) - np.exp(-Z)) / (np.exp(Z) + np.exp(-Z))
    
    # to calculate the derivative of the tanh function for the backward pass
    def derivative(self, A):
        return 1 - A**2
    
    # to be used to finetune the initialized weight if tanh function is set for the first layer
    def heuristic(self, layer_dims):
        return np.sqrt(1 / layer_dims)
    
    
# class Relu(Activation):
#     """ all the functions related to the relu activation function """
    
#     # the basic formula of the relu function for the forward pass
#     def formula(self, Z):
#         return (Z > 0) * Z
    
#     # to calculate the derivative of the relu function for the backward pass
#     def derivative(self, Z):
#         return 1 - A**2
    
#     # to be used to finetune the initialized weight if relu function is set for the first layer
#     def heuristic(self, layer_dims):
#         return np.sqrt(1 / layer_dims)    
    
#     """ define activation functions that will be needed in the DNN"""

    
#     # relu function and its derivative
#     def relu(self, Z, derivative=False):
#         if derivative:
#             return (Z > 0) * 1
#         else:
#             return (Z > 0) * Z

In [191]:
class Layer:
    """ initial class to initialize parameters and ??? """
    
    # initialize parameters
    def __init__(self, layer_b4, layer_after, activation, derivative=False):
        
#         self.L = len(layer_dim)
        
#         for i in range(1, self.L):
#         self.W = np.random.randn(layer_dim[i], layer_dim[i-1])
#         self.b = np.zeros(layer_dim[i], 1)

        self.W = np.random.randn(layer_after, layer_b4) * 0.01
        self.b = np.zeros((layer_after, 1))
        print(self.W)
        print(self.b)
        self.activation =activation
        
#         if activation == "sigmoid":
#             self.activation = sigmoid()
#         elif activation == "tanh":
#             self.activation = tanh()
#         else:
#             self.activation = relu()
        
    
    # calculate forward pass: linear fn (Z = WX + b) and non-linear (A = g(Z))
    def forward_pass(self, X):
        self.Z = np.dot(self.W, X) + self.b
        self.A = self.activation(self.Z)
        return self.Z, self.A

In [8]:
actS = Sigmoid()
print(actS.formula(3))
print(actS.derivative(2))
print(actS.heuristic(2))

actT = Tanh()
print(actT.formula(3))
print(actT.derivative(2))
print(actT.heuristic(2))

0.9525741268224334
-2
0.7071067811865476
0.9950547536867306
-3
0.7071067811865476


In [190]:
layer = Layer(2, 3)
Z, A = layer.forward_pass(X)
# Z, A = forward_pass(X)
print(Z, A)

[[-0.02112978  0.00017654]
 [ 0.00829104  0.01387959]
 [ 0.00306695  0.0101767 ]]
[[0.]
 [0.]
 [0.]]


TypeError: bad operand type for unary -: 'NoneType'

In [178]:
ds = CreateDataset()
X_size = 4
X = ds.create_X(X_size)
print(X)
Y = ds.create_Y(X)
print(Y)

[[0 1 0 0]
 [1 0 1 1]]
[1 1 1 1]


In [150]:
X_size = 4
X = np.random.randint(2, size=(2,X_size))
print(X)
print(type(X))
Y = np.sum(X, axis=0)
print(Y)
print((Y != 1))
Y[Y != 1] = 0
print(Y)

[[0 1 1 0]
 [1 0 1 1]]
<class 'numpy.ndarray'>
[1 1 2 1]
[False False  True False]
[1 1 0 1]


In [152]:
test.relu(X, derivative=True)

array([[0, 1, 1, 0],
       [1, 0, 1, 1]])

In [153]:
np.random.rand(4,3)

array([[0.87558993, 0.93684569, 0.94847332],
       [0.96739116, 0.23928585, 0.85061525],
       [0.06319912, 0.46525894, 0.21063519],
       [0.9365597 , 0.49253958, 0.06764355]])